In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j in /content/spark-3.2.3-bin-hadoop2.7\python, your SPARK_HOME may not be configured correctly

In [ ]:
#mount the content gdrive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(["EIN"],1)

application_df.head()

In [ ]:
# Determine the number of unique values in each column.
application_df.apply(lambda x:x.nunique())

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
name_count =application_df["NAME"].value_counts()
name_count

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
name_count.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_name = list(name_count[name_count<5].index)

# Replace in dataframe
for name in replace_name:
    application_df.NAME = application_df.NAME.replace(name,"Other")
    
# Check to make sure binning was successful
application_df.NAME.value_counts()

In [ ]:
#
application_count = application_df["APPLICATION_TYPE"].value_counts()
application_count

In [ ]:
#replace application_type
replace_app_count =  list(application_count[application_count<500].index)
replace_app_count

#replace dataframe
for app in replace_app_count:
  application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

#check is binning is successful or not
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

In [ ]:
replace_cls = classification_count[classification_count >1]
replace_cls

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class =list(classification_count[classification_count <100].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
application_df.dtypes

In [ ]:
# Generate our categorical variable lists
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[app_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
new_df = application_df.merge(encode_df,left_index = True,right_index = True)
new_df = new_df.drop(app_cat,1)
new_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = new_df["IS_SUCCESSFUL"].values
X = new_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=28)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 14
hidden_nodes_layer3 = 21

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation="relu"))

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation = "sigmoid"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn_new.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized_v3/",exist_ok=True)
checkpoint_path = "checkpoints_optimized_v3/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn_new.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [ ]:
#train the data
fit_model = nn_new.fit(X_train_scaled,y_train,epochs=5,callbacks=[cp_callback])

In [ ]:
# Compile the model
nn_new.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:

model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
from google.colab import files

nn_new.save('/content/AlphabetSoupCharity.h5')
files.download('/content/AlphabetSoupCharity.h5')
